# OCR XML Processing
This notebook shows the process of extracting relevant OCR information from the XML files downloaded in `load_data.ipynb` and explored in `explore_xml.ipynb`. The idea is to get the information we want into a more convenient format. For each XML file, we produce two auxiliary files: a text file containing each `TextBlock` as a continuous string and a JSON file containing information about bounding boxes.

## Imports

In [2]:
from xml.etree import ElementTree as ET # for parsing XML
from matplotlib import pyplot as plt    # for data visualization

## Globals

In [5]:
DATA_PATH = './data/'
XML_DIR = 'xml/'
TXT_DIR = 'txt/'
JSON_DIR = 'json/'

EX_PATH = './example_data/'
EX_XML_PATH = f'{EX_PATH}lccnsn830302141906-04-20ed-1seq-1.xml'

SCHEMA = '{http://schema.ccs-gmbh.com/ALTO}'

## XML Processing

In [16]:
def process_xml(filename, txt_dir, json_dir):
    block_root: ET.Element = ET.parse(EX_XML_PATH).getroot() or ET.Element('')
    for subtag in ('Layout', 'Page', 'PrintSpace'):
        block_root = block_root.find(f'{SCHEMA}{subtag}') or ET.Element('')

    with open(f'{txt_dir}{filename}.txt', 'w') as tfp, open(f'{json_dir}{filename}.json', 'w') as jfp:
        for block in block_root.findall(f'{SCHEMA}TextBlock'):
            for line in block.findall(f'{SCHEMA}TextLine'):
                
    
    return block_root

process_xml(EX_XML_PATH, './example_data/', './example_data')

<Element '{http://schema.ccs-gmbh.com/ALTO}PrintSpace' at 0x7f8167dc3450>